# Timings Inference examples 

Typical import 

In [1]:
using RoutingNetworks, TimingsInference, JLD

# Synthetic data
First, we create a synthetic network `n` (with tools from RoutingNetworks.jl). We associate it with a set of synthetic "true" timings `trueTimings`. Timings contains link times and times for each path.

Uncomment visualizations to see results

In [2]:
n = urbanNetwork(8)
# visualize(n)

# random speeds (btw 0 and 130kph)
trueTimings = randomTimings(n)

# another possibility: constant predefined speed for each road-type
trueTimings = roadTypeTimings(n)

# visualize(ShowTimes(n,trueTimings))

with path and time for all 36672 possible trips


Then, we create a synthetic set of trips based on this "true" times. This is stored as a `NetworkData` object, which contained all information for timings inference: network structure, trips and speed limits

In [3]:
# All possible trips are given (for testing purposes)
pb = perfectVirtualData(trueTimings, 0.) 

# Generate data that tries to mimic real rides in a uniform city. 
pb = noisyVirtualData(trueTimings, 0.2, 2., timeStd = 20.)

Network routing times


NetworkData: trip information in network
7169 trips (19.55% density)

## Iterative methods
The `IterativeState` object represents the state of an iterative time-estimation algorithm. It contains all the data, the current computed times, the set of paths that is used...

- `LimitedPath` is an implementation, that computes iteratively a set of paths for a subset of the trip data. A limit on the number of paths can be set.

The different methods available to compute new times are:
- `lp`: minimize MAPE (Mean absolute percentage error), uses Gurobi. `lpCo` and `lpCoNbhd` (Nbhd == Neighborhood) just add continuity constraints.
- `socp`: minimize MRE (Max ratio error), uses Mosek. `socpCo` and `Nbhd` (Nbhd == Neighborhood) just add continuity constraints.
- `mip`: implements the "minimum" constraints

In [5]:
# first, initialize the iterative method with a set of times.
# Several options here
initialTimes = uniformTimes(n,50) #speed 50kph
initialTimes = pb.minTimes # maximal allowed speed
initialTimes = randomTimes(n);

# them, create the state
s = LimitedPaths(pb, initialTimes, pathsPerTrip = 3, maxTrip=1000)

# do a number of iterations of a desired algorithm
for i = 1:5
    doIteration!(s, method="socpCoNbhd")
    println("Iteration $i, pathDiff=$(s.pathDiff)")
end
# solver args can also be given
# pathDiff measures how much the last iteration changed the path 
# (measure of convergence)

Computer
  Platform               : MACOSX/64-X86   

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 16203           
  Cones                  : 1000            
  Scalar variables       : 10920           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Conic interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Total number of eliminations : 2000
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Eliminator - elim's                 : 2000            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03 

In [7]:
s.pathDiff

1.0597